<a href="https://colab.research.google.com/github/CA-DS-SFox/MVP-network-data/blob/main/Network_Data_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1. Install libraries, functions, log into my google drive, get S3 keys

In [1]:
# install AWS SDK for python
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 10.2 MB/s 
     |████████████████████████████████| 79 kB 2.6 MB/s 
     |████████████████████████████████| 9.0 MB 36.5 MB/s 
     |████████████████████████████████| 139 kB 19.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


mount my google drive in colabs, authorise access to google sheets

In [2]:
# Mount the CA google drive
from google.colab import drive
drive.mount('/content/drive')

# authenticate my google identity
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

# libraries for working with sheets
import gspread
gc = gspread.authorize(creds)

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
import pandas as pd
import boto3

## User defined functions

Python function to load a single google sheet tab into a dataframe

### getGoogleSheet

In [4]:
# function to get the sheet from a named google source and return a dataframe
# assumes that first row in sheet is column headers

def getGoogleSheet(name_doc, name_sheet, print_indent = None):
  # open the googledoc
  try:
    # open the google-document
    sheet = gc.open(name_doc)

    # open the specific worksheet
    worksheet = sheet.worksheet(name_sheet)

  except Exception as ex:
      print ("Exception type - {0} - occurred. Arguments: {1!r}".format(type(ex).__name__, ex.args))

  # print indent 
  indent = ''
  if (isinstance(print_indent, int)):
    indent = ' ' * print_indent

  # get the data 
  ncol, nrow = worksheet.col_count, worksheet.row_count
  #print(f'Worksheet dimensions : Cols {ncol}, Rows {}, Cells {}'.format(ncol, nrow, ncol*nrow))
  print(f"{indent}Worksheet dimensions : Cols {ncol}, Rows {nrow}, Cells {ncol * nrow}")

  # worksheet.get_all_records() doesn't work it only returns 24 columns for some reason
  # cell_range is interesting, it gives you the C1R1 address of the cell origin
  # cell_range = worksheet.range('A2:CL240')
  allcells = worksheet.get_all_values()
  # get cell list dimensions
  ccol, crow = len(allcells[0]), len(allcells)
  #print('Cell list dimensions : Cols {}, Rows {}, Cells {}'.format(ccol, crow, ccol*crow))
  print(f"{indent}Cell list dimensions : Cols {ccol}, Rows {crow}, Cells {ccol * crow}")

  # make a dataframe
  df = pd.DataFrame(allcells)
  nrow, ncol = df.shape
  #print('DataFrame dimensions : Cols {}, Rows {}, Cells {}'.format(ncol, nrow, ncol*nrow))
  print(f"{indent}Dataframe dimensions : Cols {ncol}, Rows {nrow}, Cells {ncol * nrow}")

  # first row has headers
  headers = df.iloc[0]
  df = pd.DataFrame(df.values[1:], columns = headers)
  return(df)

### getEmptyOutputDF

In [5]:
def getEmptyOutputDF(preserveOriginRows = None):
    # meta data from the schema
    metacols = ['service_dataname',
              'sourcesystem', 'sourcetype', 'sourceisdownstream', 'sheetname','tabname', 'columnheader', 
              'dataowner', 'datadomain', 'datadomainitem']

    # data from the run environment and the dataset
    if preserveOriginRows == True:
      datacols = ['collection_date', 'schema_row', 'data_row', 'data_item_count','member_number', 'data']
    else:
      datacols = ['collection_date', 'data_item_count', 'member_number', 'data']

    # make a blank dataframe which will be what is returned
    df_out = pd.DataFrame(columns = metacols + datacols)
    return df_out

### collapseDataFrame

In [6]:
# for some reason this won't work when the code is part of the processGoogleSheet function
def collapseDataFrame(df_in):

  id_var_duplicates = df_in.query("data_item_count > 1").shape[0]

  print(f' ... duplicates for some variables, {id_var_duplicates} found in {df_in.shape[0]} total records, collapsing data')    
  # aggregate data from multiple entries
  cols = df_in.columns.to_list() 
  cols.remove('data') 
  df_out = df_in.groupby(cols)['data'].agg(' ; '.join).reset_index()
  print(f' ... done, {df_out.shape[0]} records remain')    

  return(df_out)


### extractFromGoogleDataFrame

In [7]:
def extractFromGoogleDataframe(df_tab_schema, testing = False):

  # ================================================================================
  # get the sheet-level details from row 0
  sheetname = df_tab_schema.iloc[0].sheetname
  tabname = df_tab_schema.iloc[0].tabname
  header_row = df_tab_schema.iloc[0].headerrow
  member_number_field = df_tab_schema.iloc[0].member_number_field

  print(f" ... Processing GoogleSheet\n{'' : >8}{'Sheet : '}{sheetname}\n{'' : >8}{'Tab : '} {tabname}")

  # ================================================================================
  # once we get to googlesheets where the data doesn't start at row 2 (python row 1) 
  # this function will need to evolve - SF_LOOK
  df_data = getGoogleSheet(sheetname, tabname, 8)
  df_out = getEmptyOutputDF()
  df_out_cols = df_out.columns
 
  # get the date and time
  outdate = str(pd.to_datetime('today'))[0:16]

  # subset for testing 
  if (testing == True):
   max_schema_rows = 1000
   max_data_rows = 2

  # ================================================================================
  # for each row of the schema dataframe
  for schemaindex, schemarow in df_tab_schema.iterrows():
    collection_column = schemarow['columnheader']

    # transfer the source information
    df_out_row_template = schemarow
    df_out_row_template['collection_date'] = outdate
    df_out_row_template['schema_row'] = schemaindex

    if testing == True and schemaindex > max_schema_rows:
      break

    # for each row of the data
    for dataindex, datarow in df_data.iterrows():
      if testing == True and dataindex > max_data_rows:
        break

      # get the member_number 
      member_number = datarow[member_number_field]

      # naive check, this can be improved - SF_LOOK
      if '/' in member_number:
        dataitem = datarow[collection_column]

        df_out_row = df_out_row_template
        df_out_row['data_row'] = dataindex
        df_out_row['member_number'] = member_number
        df_out_row['data'] = dataitem
        df_out = df_out.append(df_out_row, ignore_index = True)

  # ================================================================================
  variable_counts = df_out.groupby(['member_number', 'service_dataname']).size().reset_index(name = 'data_item_count')
  df_out.set_index(['member_number', 'service_dataname'], inplace=True)
  df_out.update(variable_counts.set_index(['member_number', 'service_dataname']))
  df_out.reset_index(inplace=True)
  df_out = df_out.reset_index()

  # ================================================================================
  # final result
  df_out = df_out[df_out_cols]
  return(df_out)

### processGoogleSheet

In [31]:
# each time we hit a new tab in the schema sheet, do this
# which returns the data from the google sheet

def processGoogleSheet(df_tab_schema, testing = False, checking = True):

  df_out = extractFromGoogleDataframe(df_tab_schema, testing = testing)
  
  # report any member_numbers with more than one data value
  id_var_duplicates = df_out.query("data_item_count > 1").shape[0]
  if checking & id_var_duplicates > 0:
    df_out = collapseDataFrame(df_out)

  return(df_out)

### get S3 keys

In [9]:
# get S3 information
BUCKET_NAME = 'network-data-service-development' 

# get the keys
df_s3 = getGoogleSheet('secrets','Info')
S3_Key = df_s3.iat[0,1]
S3_Secret = df_s3.iat[1,1]

Worksheet dimensions : Cols 26, Rows 1001, Cells 26026
Cell list dimensions : Cols 2, Rows 3, Cells 6
Dataframe dimensions : Cols 2, Rows 3, Cells 6


# Step 2. Process the Network Data Schema tab and collect the data

In [32]:
# iterate the rows of the schema sheet
network_schema_sheet = 'Network data service sources'
network_schema_tab = 'Network Data Service Schema'
print(f' ... Getting the Data Schema from Sheet {network_schema_sheet}, Tab {network_schema_tab}')
df_schema = getGoogleSheet(network_schema_sheet, network_schema_tab)

# This is the long format data that will be created
df_tidy = getEmptyOutputDF()

current_source = ''

for index, row in df_schema.iterrows():
    current_tab = row['tabname']

    # data source reference has changed
    if current_tab != current_source:
      current_source = current_tab
      current_sheet = row['sheetname']

      print(f'\n ... changing data source to : {current_sheet}, {current_tab}')
  
      df_tab_schema = df_schema.query(f'sheetname == "{current_sheet}" & tabname == "{current_tab}"')

      # -- add something in here to deal with multiple entries for a member_number
      df_tab_data = processGoogleSheet(df_tab_schema)
      df_tidy = pd.concat([df_tidy, df_tab_data], ignore_index = True)
      df_tidy.reset_index(drop=True, inplace=True)

 ... Getting the Data Schema from Sheet Network data service sources, Tab Network Data Service Schema
Worksheet dimensions : Cols 34, Rows 994, Cells 33796
Cell list dimensions : Cols 12, Rows 20, Cells 240
Dataframe dimensions : Cols 12, Rows 20, Cells 240

 ... changing data source to : Parameters, Offices
 ... Processing GoogleSheet
        Sheet : Parameters
        Tab :  Offices
        Worksheet dimensions : Cols 22, Rows 1001, Cells 22022
        Cell list dimensions : Cols 3, Rows 273, Cells 819
        Dataframe dimensions : Cols 3, Rows 273, Cells 819

 ... changing data source to : MBR - Member Number - Member Name, MBR-Member_No-Name
 ... Processing GoogleSheet
        Sheet : MBR - Member Number - Member Name
        Tab :  MBR-Member_No-Name
        Worksheet dimensions : Cols 26, Rows 1000, Cells 26000
        Cell list dimensions : Cols 4, Rows 264, Cells 1056
        Dataframe dimensions : Cols 4, Rows 264, Cells 1056

 ... changing data source to : Member + Location 

# Step 3. Have a look at what's been created

In [15]:
df.query("service_dataname == 'member_MBR' & data_item_count > 1")

,service_dataname,sourcesystem,sourcetype,sourceisdownstream,sheetname,tabname,columnheader,dataowner,datadomain,datadomainitem,collection_date,data_item_count,member_number,data
515,member_MBR,Okta,googlesheet,1,MBR - Member Number - Member Name,MBR-Member_No-Name,MBR,Simon Sheridan,Advice Network,member,2022-07-15 13:47,2.0,85/0050,MBR_Arun_Chichester ; MBR_Ashfield


In [11]:
df = df_tidy
df[(df['member_number'] == '85/0050')]

,service_dataname,sourcesystem,sourcetype,sourceisdownstream,sheetname,tabname,columnheader,dataowner,datadomain,datadomainitem,collection_date,data_item_count,member_number,data
226,member_MBR_old,Okta,googlesheet,1,Parameters,Offices,office,Suzanne Fox,Advice Network,member,2022-07-15 13:47,1.0,85/0050,MBR_Arun_Chichester
515,member_MBR,Okta,googlesheet,1,MBR - Member Number - Member Name,MBR-Member_No-Name,MBR,Simon Sheridan,Advice Network,member,2022-07-15 13:47,2.0,85/0050,MBR_Arun_Chichester ; MBR_Ashfield
748,member_LA,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),Local_Authority__c,Tom Williams,Geographical,local authority,2022-07-15 13:47,1.0,85/0050,0014K000009EGflQAG
1014,member_address,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),BillingAddress.street,Tom Williams,Geographical,memberlocation,2022-07-15 13:47,1.0,85/0050,"Town Hall,Clarence Road,"
1280,member_charity_number,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),Charity_Number__c,Tom Williams,Advice Network,member,2022-07-15 13:47,1.0,85/0050,1099640
1546,member_city,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),BillingAddress.city,Tom Williams,Geographical,memberlocation,2022-07-15 13:47,1.0,85/0050,BOGNOR REGIS
1812,member_company_number,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),Company_Number__c,Tom Williams,Advice Network,member,2022-07-15 13:47,1.0,85/0050,4787378
2078,member_end_date,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),End_Date__c,Tom Williams,Advice Network,member,2022-07-15 13:47,1.0,85/0050,
2344,member_lat,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),BillingAddress.latitude,Tom Williams,Geographical,geo coordinates,2022-07-15 13:47,1.0,85/0050,50.78317
2610,member_lon,Salesforce,googlesheet,1,Member + Location sheet (Salesforce data),Member List (1),BillingAddress.longitude,Tom Williams,Geographical,geo coordinates,2022-07-15 13:47,1.0,85/0050,-0.67037


# Step 4. Make useful subsets

In [33]:
df_tidy.head()

,service_dataname,sourcesystem,sourcetype,sourceisdownstream,sheetname,tabname,columnheader,dataowner,datadomain,datadomainitem,collection_date,data_item_count,member_number,data
0,member_MBR_old,Okta,googlesheet,1,Parameters,Offices,office,Suzanne Fox,Advice Network,member,2022-07-15 14:10,1.0,15/0002,MBR_Harrow
1,member_MBR_old,Okta,googlesheet,1,Parameters,Offices,office,Suzanne Fox,Advice Network,member,2022-07-15 14:10,1.0,15/0003,MBR_Enfield
2,member_MBR_old,Okta,googlesheet,1,Parameters,Offices,office,Suzanne Fox,Advice Network,member,2022-07-15 14:10,1.0,15/0010,MBR_Camden
3,member_MBR_old,Okta,googlesheet,1,Parameters,Offices,office,Suzanne Fox,Advice Network,member,2022-07-15 14:10,1.0,15/0016,MBR_Barnet
4,member_MBR_old,Okta,googlesheet,1,Parameters,Offices,office,Suzanne Fox,Advice Network,member,2022-07-15 14:10,1.0,15/0026,MBR_East_End


In [70]:
# =======================================================================================
# Network data fields to extract
service_dataname_fields = ['member_MBR', 'member_name', 'member_salesforce_id', 'member_status']
# get the data subset
df_flat = df_tidy[df_tidy['service_dataname'].isin(service_dataname_fields)]
# data columns to extract for those data fields
flat_columns = ['member_number','service_dataname','data']
df_flat = df_flat[flat_columns]

# =======================================================================================
# wide version of the data
df_wide = (df_flat.pivot(index=['member_number'], columns="service_dataname", values="data").sort_index(level = [1, 0]))
#df_wide = (df_flat.pivot(index=['member_number', 'service_dataname'], columns="service_dataname", values="data"))

# =======================================================================================
# meta data for these fields
meta_columns = ['service_dataname', 'dataowner','datadomain']
df_meta = df_tidy[df_tidy['service_dataname'].isin(service_dataname_fields)]
df_meta[meta_columns].drop_duplicates()

,service_dataname,dataowner,datadomain
264,member_MBR,Simon Sheridan,Advice Network
519,member_salesforce_id,Tom Williams,Advice Network
1583,member_status,Tom Williams,Advice Network
1849,member_name,Tom Williams,Advice Network


In [71]:
print(df_flat.shape)
print(df_flat.drop_duplicates().shape)
print(df_flat.value_counts('member_number'))

#df_wide = (df_flat.pivot(index=['member_number'], columns="service_dataname"))['data']

df_flat.query("member_number == '25/0028'")

(1053, 3)
(1053, 3)
member_number
15/0002    4
75/0037    4
70/0012    4
70/0014    4
70/0015    4
          ..
25/0011    3
25/0024    3
75/0058    3
75/0052    3
95/0037    3
Length: 266, dtype: int64


,member_number,service_dataname,data
401,25/0028,member_MBR,MBR_Mid-North_Yorkshire
617,25/0028,member_salesforce_id,0014K00000PcCB3QAN
1681,25/0028,member_status,
1947,25/0028,member_name,Citizens Advice Mid-North Yorkshire


In [74]:
df_wide.to_csv('wide.csv')

In [20]:
df_meta.head()

,service_dataname,dataowner,datadomain
0,member_MBR_old,Suzanne Fox,Advice Network
262,member_MBR,Simon Sheridan,Advice Network
2644,member_name,Tom Williams,Advice Network
3442,member_salesforce_id,Tom Williams,Advice Network


# Step 4. Write to S3

In [ ]:
df_raw = getGoogleSheet('Live* Network Dashboard', 'Primary Service Areas by LA')
df_raw.to_csv('test.csv')

Worksheet dimensions : Cols 28, Rows 1000, Cells 28000
Cell list dimensions : Cols 14, Rows 336, Cells 4704
DataFrame dimensions : Cols 14, Rows 336, Cells 4704


In [ ]:
session = boto3.Session(
  aws_access_key_id = S3_Key,
  aws_secret_access_key = S3_Secret
)

#Creating S3 Resource From the Session.
s3 = session.resource('s3')

file_name = 'test.csv'
object_name = 'testx.csv'

object = s3.Object(BUCKET_NAME, object_name)

result = object.put(Body=open('test.csv', 'rb'))
res = result.get('ResponseMetadata')

if res.get('HTTPStatusCode') == 200:
    print('File Uploaded Successfully')
else:
    print('File Not Uploaded')

File Uploaded Successfully


# Testing section

In [221]:
# get a specified tabs data
current_sheet = 'Casebook - OKTA Accounts'
current_tab = 'Casebook and OKTA'

print(f'\n ... changing data source to : {current_sheet}, {current_tab}')

df_tab_schema = df_schema[(df_schema['tabname'] == current_tab )]
df_tab_schema.query(f'sheetname == "{current_sheet}" & tabname == "{current_tab}"')
df_tab_data = processGoogleSheet(df_tab_schema, checking = True)



 ... changing data source to : Casebook - OKTA Accounts, Casebook and OKTA
 ... Processing GoogleSheet
        Sheet : Casebook - OKTA Accounts
        Tab :  Casebook and OKTA
        Worksheet dimensions : Cols 25, Rows 999, Cells 24975
        Cell list dimensions : Cols 11, Rows 552, Cells 6072
        Dataframe dimensions : Cols 11, Rows 552, Cells 6072
 ... duplicates for some variables, 8 found in 550 total records, collapsing data
 ... done, 546 records remain


In [227]:
df_tab_data
df_tab_data.query("data_item_count > 1")

,service_dataname,sourcesystem,sourcetype,sourceisdownstream,sheetname,tabname,columnheader,dataowner,datadomain,datadomainitem,collection_date,data_item_count,member_number,data
271,member_MBR,Okta,googlesheet,1,Casebook - OKTA Accounts,Casebook and OKTA,MBR_,Simon Sheridan,Advice Network,member,2022-07-15 13:38,2.0,35/0039,MBR_Bury_Bolton ;
272,member_MBR,Okta,googlesheet,1,Casebook - OKTA Accounts,Casebook and OKTA,MBR_,Simon Sheridan,Advice Network,member,2022-07-15 13:38,2.0,85/0056,MBR_Central_Dorset ;
544,member_casebook_name,Okta,googlesheet,1,Casebook - OKTA Accounts,Casebook and OKTA,Casebook Name,Simon Sheridan,Advice Network,member,2022-07-15 13:38,2.0,35/0039,Bury District (member) ; Bolton & District (me...
545,member_casebook_name,Okta,googlesheet,1,Casebook - OKTA Accounts,Casebook and OKTA,Casebook Name,Simon Sheridan,Advice Network,member,2022-07-15 13:38,2.0,85/0056,Weymouth & Portland (member) ; Citizens Advice...


# Appendix - supporting methods

In [244]:
# get a specified tabs data
current_sheet = 'Parameters'
current_tab = 'Offices'

print(f'\n ... changing data source to : {current_sheet}, {current_tab}')

df_tab_schema = df_schema[(df_schema['tabname'] == current_tab )]
df_tab_schema.query(f'sheetname == "{current_sheet}" & tabname == "{current_tab}"')
df_tab_data = processGoogleTabSchema(df_tab_schema)



 ... changing data source to : Parameters, Offices
 ... Processing GoogleSheet
        Sheet : Parameters
        Tab :  Offices
        Worksheet dimensions : Cols 22, Rows 1001, Cells 22022
        Cell list dimensions : Cols 3, Rows 273, Cells 819
        Dataframe dimensions : Cols 3, Rows 273, Cells 819


In [ ]:
# print the names of all the tabs in a sheet
#client = gspread.authorize(creds)
#sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1xXt0yvkvn5yh28N0rdHuPJEQvLmLoj3qh8ZDX5obgaw/edit#gid=1587992043') 

#flag = True
#i = 0

#while flag:
#    try:
#        ith = sheet.get_worksheet(i)
#        i += 1
#        print(ith)
#    except Exception as e:
#        print(e)
#        flag = False
#
#print(f'Total worksheet = {i}')


<Worksheet 'Dashboard' id:1989303307>
<Worksheet 'Vol Data' id:1557786994>
<Worksheet 'Network Dashboard' id:1536928015>
<Worksheet 'RM Intel' id:1845792682>
<Worksheet 'INDEX' id:233898872>
<Worksheet 'Income data' id:1941024203>
<Worksheet 'LCA Staff' id:1905672712>
<Worksheet 'Contact Sheet' id:1364673288>
<Worksheet 'Mirror LPQF' id:1326627537>
<Worksheet 'Maps - Mirror' id:773535940>
<Worksheet 'Exemptions - mirror' id:1208173255>
<Worksheet 'Core Service - mirror' id:1930923112>
<Worksheet 'ICS & LCA' id:1813462983>
<Worksheet 'SCC Live Referrals - Mirror' id:1999676966>
<Worksheet 'RM intel Mergers' id:116970853>
<Worksheet 'PA tracker LCA tab - mirror' id:516028987>
<Worksheet 'Clients' id:748898763>
<Worksheet 'FHM - Risk table' id:1531961885>
<Worksheet 'FHM - Mapping table' id:2143724658>
<Worksheet 'FHM - WIP Reserves' id:1710937234>
<Worksheet 'WIP - Supply Chain view' id:1559736093>
<Worksheet 'Mirror Funding' id:958806554>
<Worksheet 'Mirror - Network Tracker' id:6356128

KeyboardInterrupt: ignored